利用gurobi中的三种数据结构Multidict/Tuplelist/Tupledict解决网络流问题

应用场景如下：
有两个城市A、B，生产了两种商品com1、com2，必须装运到三个城市C、D、E。其中网络的每一条弧都有容量限制以及运输成本

In [150]:
import gurobipy as grb

# 2种商品
commodities = ['com1','com2']

# 2个产地和3个目的地
nodes = ['A','B','C','D','E']

# 网络中每条弧的容量，使用Multidict一次性创建多个字典，可迭代
# arcs为键，capacity为值
arcs, capacity = grb.multidict(
{
    ('A','C'): 100,
    ('A','D'): 80,
    ('A','E'): 120,
    ('B','C'): 120,
    ('B','D'): 120,
    ('B','E'): 120
})

# 商品在不同弧上的运输成本，是tupledict形式(不同于Multidict)，其键在内部存储格式是tuplelist，可以用select/sum/prod等加快变量选取运算
cost = {
    ('com1','A','C'): 10,
    ('com1','A','D'): 20,
    ('com1','A','E'): 60,
    ('com1','B','C'): 40,
    ('com1','B','D'): 40,
    ('com1','B','E'): 30,
    ('com2','A','C'): 20,
    ('com2','A','D'): 20,
    ('com2','A','E'): 80,
    ('com2','B','C'): 60,
    ('com2','B','D'): 70,
    ('com2','B','E'): 30
}

# 商品在不同节点的流入量、流出量/需求量，同样是tupledict形式，可以用select/sum/prod等加快变量选取运算
# 正数表示产量、负数表示需求量
inflow = {
    ('com1','A'): 50,
    ('com1','B'): 60,
    ('com1','C'): -50,
    ('com1','D'): -50,
    ('com1','E'): -10,
    ('com2','A'): 60,
    ('com2','B'): 40,
    ('com2','C'): -40,
    ('com2','D'): -30,
    ('com2','E'): -30
}

In [151]:
# 创建模型
m = grb.Model('netflow')

# 创建变量
# flow是tupledict类型的变量，可以使用select快速筛选，键是('com1','A','C')格式，值是cost
flow = m.addVars(commodities, arcs, obj=cost, name='flow')

# 添加容量约束
# 使用迭代表达式，i表示产地，j表示目的地
# flow.sum('*',i,j)表示对i->j的所有不同商品的总量求和
m.addConstrs((flow.sum('*',i,j) <= capacity[i,j] for i,j in arcs), 'cap')

# 添加节点流入=流出+需求量的约束
# h表示商品，j表示节点，包括产地和目的地
# flow.sum(h,'*',j)表示商品h经所有中间节点到达j后的总数量
# flow.sum(h,j,'*')表示商品h从j节点流出去的总数量
m.addConstrs((flow.sum(h,'*',j) + inflow[h,j] == flow.sum(h,j,'*') for h in commodities for j in nodes), 'node')

# 求解模型
m.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 16 rows, 12 columns and 36 nonzeros
Model fingerprint: 0xc43e5943
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+02]
Presolve removed 16 rows and 12 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.5000000e+03   0.000000e+00   2.000000e+01      0s
Extra simplex iterations after uncrush: 1
       1    5.5000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.02 seconds
Optimal objective  5.500000000e+03


In [152]:
# 输出结果
if m.status == grb.GRB.Status.OPTIMAL:
    solution = m.getAttr('x',flow)
    for h in commodities:
        print('\nOptimal flows for %s:' % h)
        for i,j in arcs:
            if solution[h,i,j] > 0:
                print('%s -> %s: %g' % (i,j,solution[h,i,j]))


Optimal flows for com1:
A -> C: 50
B -> D: 50
B -> E: 10

Optimal flows for com2:
A -> C: 30
A -> D: 30
B -> C: 10
B -> E: 30
